### 不使用主动空间近似优化的前提下
#### 目的:选取30%的UCCD 和70%的UCCGD 构成算符池

In [1]:
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper, QubitConverter
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer,FreezeCoreTransformer
import numpy as np
from qiskit.algorithms.optimizers import SLSQP
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.primitives import Estimator
from qiskit.algorithms.minimum_eigensolvers import AdaptVQE
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

from qiskit.circuit import QuantumCircuit
from copy import  deepcopy
from qiskit.circuit import Parameter,ParameterVector
import sys
sys.path.append('..')
from AdaptVQE_.My_adapt import MyG_AdaotVQE,MyAdaptVQE
# Define Molecule
molecule = MoleculeInfo(
    ["H", "H","H","H","H","H"], [(0.0, 0.0, 0.0), (0.8*1, 0.0, 0.0), (0.8*2, 0.0, 0.0), (0.8*3, 0.0, 0.0), (0.8*4, 0.0, 0.0), (0.8*5, 0.0, 0.0)],
    multiplicity=1,  # = 2*spin + 1
    charge=0,
)
driver = PySCFDriver().from_molecule(molecule)

problem = driver.run()
converter = QubitConverter(JordanWignerMapper(), two_qubit_reduction=True)
transformer = FreezeCoreTransformer(
    freeze_core=True,  # how many electrons we have in our active space
    # how many orbitals we have in our active space
    remove_orbitals=[-3, -2]
)
#problem = transformer.transform(problem=problem)

In [8]:
from qiskit_nature.second_q.circuit.library import UCC
from qiskit.circuit.library import EvolvedOperatorAnsatz
uccsd = UCC(num_spatial_orbitals=problem.num_spatial_orbitals,num_particles=problem.num_particles,excitations='sd',\
    generalized=False)
uccgsd = UCC(num_spatial_orbitals=problem.num_spatial_orbitals,num_particles=problem.num_particles,excitations='sd',\
    generalized=True)
uccgd = UCC(num_spatial_orbitals=problem.num_spatial_orbitals,num_particles=problem.num_particles,excitations='d',\
    generalized=True)

uccsd_secondop = [converter.convert(i) for i in uccsd.excitation_ops()] #117
uccgsd_secondop = [converter.convert(i) for i in uccgsd.excitation_ops()] #285
uccgd_secondop = [converter.convert(i) for i in uccgd.excitation_ops()] #255
##
uccsd_secondop_pool = [EvolvedOperatorAnsatz(operators=i, insert_barriers=True, name='SD_'+str(index), parameter_prefix='SD_'+str(index)).to_instruction()\
                                                for index, i in enumerate(uccsd_secondop)]
uccgsd_secondop_pool = [EvolvedOperatorAnsatz(operators=i, insert_barriers=True, name='GSD_'+str(index), parameter_prefix='GSD_'+str(index)).to_instruction()\
                                                for index, i in enumerate(uccgsd_secondop)]
uccgd_secondop_pool = [EvolvedOperatorAnsatz(operators=i, insert_barriers=True, name='GD_'+str(index), parameter_prefix='GD_'+str(index)).to_instruction()\
                                                for index, i in enumerate(uccgd_secondop)]



In [14]:
k = [i for i in uccsd_secondop if i not in uccgsd_secondop]

[PauliSumOp(SparsePauliOp(['IIIIIIIYYIXY', 'IIIIIIIXXIXY', 'IIIIIIIXYIYY', 'IIIIIIIYXIYY', 'IIIIIIIXYIXX', 'IIIIIIIYXIXX', 'IIIIIIIYYIYX', 'IIIIIIIXXIYX'],
               coeffs=[ 0.125+0.j, -0.125+0.j, -0.125+0.j, -0.125+0.j,  0.125+0.j,  0.125+0.j,
   0.125+0.j, -0.125+0.j]), coeff=1.0),
 PauliSumOp(SparsePauliOp(['IIIIIIYZYIXY', 'IIIIIIXZXIXY', 'IIIIIIXZYIYY', 'IIIIIIYZXIYY', 'IIIIIIXZYIXX', 'IIIIIIYZXIXX', 'IIIIIIYZYIYX', 'IIIIIIXZXIYX'],
               coeffs=[ 0.125+0.j, -0.125+0.j, -0.125+0.j, -0.125+0.j,  0.125+0.j,  0.125+0.j,
   0.125+0.j, -0.125+0.j]), coeff=1.0),
 PauliSumOp(SparsePauliOp(['IIIIIIIYYXZY', 'IIIIIIIXXXZY', 'IIIIIIIXYYZY', 'IIIIIIIYXYZY', 'IIIIIIIXYXZX', 'IIIIIIIYXXZX', 'IIIIIIIYYYZX', 'IIIIIIIXXYZX'],
               coeffs=[ 0.125+0.j, -0.125+0.j, -0.125+0.j, -0.125+0.j,  0.125+0.j,  0.125+0.j,
   0.125+0.j, -0.125+0.j]), coeff=1.0),
 PauliSumOp(SparsePauliOp(['IIIIIIYZYXZY', 'IIIIIIXZXXZY', 'IIIIIIXZYYZY', 'IIIIIIYZXYZY', 'IIIIIIXZYXZX', 'IIIIIIYZXXZX', 'III

#### 筛选前X名的UCCD算符块
使用hf态作为初始态 计算和接下来的所有算符的下降梯度 选取更大的入选  
STEP 1:找到UCCD UCCGD UCCSD 中梯度比较大的算符 (标准？)

In [2]:
a = MyAdaptVQE(ES_problem=problem)

开始初始化:挑选第一个算符...
初始化结果:第87项被选定,此项梯度最大


In [3]:
a.hamiltonian

PauliSumOp(SparsePauliOp(['IIIIIIIIIIII', 'IIIIIIIIIIIZ', 'IIIIIIIIIYZY', 'IIIIIIIIIXZX', 'IIIIIIIYZZZY', 'IIIIIIIXZZZX', 'IIIIIIIIIIZI', 'IIIIIIIIYZYI', 'IIIIIIIIXZXI', 'IIIIIIYZZZYI', 'IIIIIIXZZZXI', 'IIIIIIIIIZII', 'IIIIIIIYZYII', 'IIIIIIIXZXII', 'IIIIIIIIZIII', 'IIIIIIYZYIII', 'IIIIIIXZXIII', 'IIIIIIIZIIII', 'IIIIIIZIIIII', 'IIIIIZIIIIII', 'IIIYZYIIIIII', 'IIIXZXIIIIII', 'IYZZZYIIIIII', 'IXZZZXIIIIII', 'IIIIZIIIIIII', 'IIYZYIIIIIII', 'IIXZXIIIIIII', 'YZZZYIIIIIII', 'XZZZXIIIIIII', 'IIIZIIIIIIII', 'IYZYIIIIIIII', 'IXZXIIIIIIII', 'IIZIIIIIIIII', 'YZYIIIIIIIII', 'XZXIIIIIIIII', 'IZIIIIIIIIII', 'ZIIIIIIIIIII', 'IIIIIIIIIIZZ', 'IIIIIIIIYZYZ', 'IIIIIIIIXZXZ', 'IIIIIIYZZZYZ', 'IIIIIIXZZZXZ', 'IIIIIIIIIZIZ', 'IIIIIIIYZYIZ', 'IIIIIIIXZXIZ', 'IIIIIIIIZIIZ', 'IIIIIIYZYIIZ', 'IIIIIIXZXIIZ', 'IIIIIIIZIIIZ', 'IIIIIIZIIIIZ', 'IIIIIIIIIYIY', 'IIIIIIIIIXIX', 'IIIIIIIYZZIY', 'IIIIIIIXZZIX', 'IIIIIIIIYXXY', 'IIIIIIIIYYYY', 'IIIIIIIIXXYY', 'IIIIIIIIYYXX', 'IIIIIIIIXXXX', 'IIIIIIIIXYYX', 'IIIIIIYZZXXY'

In [ ]:
from qiskit.circuit.library import EvolvedOperatorAnsatz
operator_pool = [EvolvedOperatorAnsatz(i,) for i in a.operator_pool_femonic_op]
b = MyAdaptVQE(custom_operation_pool=a.operator_pool_femonic_op)

In [5]:
len(a.operator_pool_femonic_op)
len(a.UCCGD_index)

24

In [7]:
k=0
for i in a.UCCGD_index:
    if a.UCCGD_op[i] in a.operator_pool_femonic_op:
        print(f'已经{a.UCCGD_op[i]}在算符池里了')
        k+=1
print(k)

已经-0.125 * YZZZYIXZZZYI
- 0.125 * XZZZYIYZZZYI
- 0.125 * XZZZXIXZZZYI
+ 0.125 * YZZZXIYZZZYI
- 0.125 * XZZZYIXZZZXI
+ 0.125 * YZZZYIYZZZXI
+ 0.125 * YZZZXIXZZZXI
+ 0.125 * XZZZXIYZZZXI在算符池里了
已经-0.125 * YZZZYIIIIXYI
- 0.125 * XZZZYIIIIYYI
- 0.125 * XZZZXIIIIXYI
+ 0.125 * YZZZXIIIIYYI
- 0.125 * XZZZYIIIIXXI
+ 0.125 * YZZZYIIIIYXI
+ 0.125 * YZZZXIIIIXXI
+ 0.125 * XZZZXIIIIYXI在算符池里了
已经-0.125 * IIIYYIXZZZYI
- 0.125 * IIIXYIYZZZYI
- 0.125 * IIIXXIXZZZYI
+ 0.125 * IIIYXIYZZZYI
- 0.125 * IIIXYIXZZZXI
+ 0.125 * IIIYYIYZZZXI
+ 0.125 * IIIYXIXZZZXI
+ 0.125 * IIIXXIYZZZXI在算符池里了
已经-0.125 * IIYZYIIIXZYI
- 0.125 * IIXZYIIIYZYI
- 0.125 * IIXZXIIIXZYI
+ 0.125 * IIYZXIIIYZYI
- 0.125 * IIXZYIIIXZXI
+ 0.125 * IIYZYIIIYZXI
+ 0.125 * IIYZXIIIXZXI
+ 0.125 * IIXZXIIIYZXI在算符池里了
已经-0.125 * IYZZYIIXZZYI
- 0.125 * IXZZYIIYZZYI
- 0.125 * IXZZXIIXZZYI
+ 0.125 * IYZZXIIYZZYI
- 0.125 * IXZZYIIXZZXI
+ 0.125 * IYZZYIIYZZXI
+ 0.125 * IYZZXIIXZZXI
+ 0.125 * IXZZXIIYZZXI在算符池里了
已经-0.125 * IIIYZYIIIXZY
- 0.125 * IIIXZYIIIYZ

In [10]:
len(a.operator_pool_femonic_op)

26

In [ ]:
###UCCGD的所有算符 第一步的下降梯度 
n = uccgd_ansatz.init_state_hf.num_qubits
pre_circuit=[]
for i in uccgd_ansatz.excitation_pool_instruction:
    qc = QuantumCircuit(n)
    qc.append(uccgd_ansatz.init_state_hf,range(n))
    qc.append(i,range(n))
    pre_circuit.append(qc)

estimator = Estimator()
job = estimator.run(circuits=pre_circuit,observables=uccgd_ansatz.commutors,parameter_values=[[0.0]]*len(pre_circuit))
result = job.result()


In [ ]:
abs_value = np.abs(result.values)
np_sort = np.argsort(np.abs(result.values))
np_sort = np_sort[::-1]
#print(np.abs(result.values))
print(f'{np_sort}')
for index,i in enumerate(np_sort[0:20]):
    print(f'第{index}项的值为{abs_value[i]}')

In [ ]:
result.values[189] #-0.25020478036677607

In [ ]:
len(d4)
len(difference4)

In [ ]:
uccd_ansatz.excitation_pool[0].primitive.paulis

In [ ]:
2e-3

In [ ]:
n = uccgd_ansatz.init_state_hf.num_qubits
pre_circuit=[]
for i in uccgd_ansatz.excitation_pool_instruction:
    qc = QuantumCircuit(n)
    qc.append(uccd_ansatz.init_state_hf,range(n))
    qc.append(i,range(n))
    pre_circuit.append(qc)

estimator = Estimator()
job = estimator.run(circuits=pre_circuit,observables=uccd_ansatz.commutors,parameter_values=[[0.0]]*len(pre_circuit))
result = job.result()


In [ ]:
rank_index = np.argsort(np.abs(result.values)) #从小到大排列的
len(rank_index) #76
rank_index

In [ ]:
len(uccsd_ansatz.excitation_pool) #92
len(uccgsd_ansatz.excitation_pool) #285
len(uccgd_ansatz.excitation_pool) #255
len(uccd_ansatz.excitation_pool) #76

In [ ]:
print(rank_index1 == rank_index2)

In [ ]:
rank_index1 = np.argsort(result.values)

In [ ]:
rank = np.argsort(np.abs(val))
len(rank[0])

In [ ]:
rank = np.argsort(val[0])

In [ ]:
print(f'aaa{3==3}')

In [ ]:
uccsd.excitation_ops()

In [ ]:
1 in [1,2,3] 